# Run GriddedCCP

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
    
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [5]:
args_32_ccp = parse_arguments("SelfAttnCNP zs-multi-mnist --max-epochs 1 --batch-size 6 --name test --chckpnt-dirname results/test/ --is-progressbar".split())


In [6]:
args_64_anp = [parse_arguments("SelfAttnCNP {}  --batch-size 6 --chckpnt-dirname results/test/ ".format(d).split())
                for d in ["zs-multi-mnist"]]

executor_64_anp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=1)
executor_64_anp.update_parameters(num_gpus=1, 
                                   time=10,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                  constraint="volta32gb",
                                  partition="dev"
                                  )

job_64_anp = executor_64_anp.map_array(Run(), args_64_anp)

In [19]:
job_64_anp


[SlurmJob<job_id=17872513_0, task_id=0, state="FAILED">]

In [17]:
print(job_64_anp[0].stderr())

INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=256 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
submitit WARNING (2019-09-19 16:33:55,614) - Caught USR1 signal 10 on learnfair0801: this job is timed-out.
INFO:submitit:Calling checkpoint method.
INFO:submitit:Requeued job 17872513_0 (0 remaining timeouts)
INFO:submitit:Exiting gracefully after preemption/timeout.
slurmstepd-learnfair0801: error: *** JOB 17872513 ON learnfair0801 CANCELLED AT 2019-09-19T16:33:56 DUE TO JOB REQUEU

In [12]:
print(job_64_anp[0].stdout())

submitit INFO (2019-09-19 16:26:00,353) - Starting with JobEnvironment(job_id=17872513_0, hostname=learnfair0801, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2019-09-19 16:26:00,353) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17872513_0/17872513_0_submitted.pkl

